![TQ42%20Banner.png](attachment:TQ42%20Banner.png)

# Welcome to TQ42

## Example: Using Custom Quantum Layers

## ✨ Introduction 
In this notebook, we will cover how to apply the custom quantum layer within a custom model architecture in the SDK. <br>

Here we will train a custom time series prediction problem using a custom quantum layer with a classical dense layer. For more details, please see the [developer documentation](https://docs.tq42.com/en/latest/index.html).

## 🎯 Objective

Applying the custom quantum layer within a custom model architecture involves:
1. Do this
2. then this
3. finally this

In [ ]:
# Import packages

from google.protobuf.json_format import MessageToDict
from tq42.client import TQ42Client
from tq42.experiment_run import ExperimentRun
from tq42.compute import HardwareProto
from tq42.algorithm import (
    TrainDataProcessingParametersProto,
    OptimProto,
    LossFuncProto,
    DatasetStorageInfoProto,
    GenericMLTrainMetadataProto,
    GenericMLTrainParametersProto,
    Layer,
    ClassicalDenseLayer,
    MLModelType,
    TrainModelInfoProto,
    MLTrainInputsProto,
    AlgorithmProto,
    MeasureProto,
    CustomQuantumLayer,
    CnotGate,
    HadamardGate,
    VariationalGate,
    EncodingGate,
    MeasurementGate,
    Gate
)


In [ ]:
# Connect to the cloud service
# https://terraquantum.io

with TQ42Client() as client:
    client.login()

In [ ]:


#Here is an example of how to apply the custom quantum layer within a custom model architecture in the SDK.
#The following example trains a custom time series prediction problem using a custom quantum layer with a classical dense layer.

custom_quantum_layer_msg = CustomQuantumLayer(
    num_qubits=2,
    gates=[
        Gate(hadamard=HadamardGate(wire=0)),
        Gate(hadamard=HadamardGate(wire=1)),
        Gate(variational=VariationalGate(wire=0, rotation=MeasureProto.X)),
        Gate(
            encoding=EncodingGate(wire=1, rotation=MeasureProto.Y, feature_id=0)
        ),
        Gate(cnot=CnotGate(wire1=0, wire2=1)),
        Gate(variational=VariationalGate(wire=1, rotation=MeasureProto.X)),
        Gate(measurement=MeasurementGate(wire=0, pauli=MeasureProto.X)),
        Gate(measurement=MeasurementGate(wire=1, pauli=MeasureProto.X)),
    ],
)

env_msg = GenericMLTrainMetadataProto(
    parameters=GenericMLTrainParametersProto(
        model_type=MLModelType.MLP,
        # Add and customize layers here
        layers=[
            Layer(custom_quantum_layer=custom_quantum_layer_msg),
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=1, bias=True)),
        ],
        # WHY ONLY 1 EPOCH?
        num_epochs=1,
        k_fold=1,
        batch_size=128,
        learning_rate=0.01,
        optim=OptimProto.ADAM,
        loss_func=LossFuncProto.MAE,
        train_model_info=TrainModelInfoProto(
            # Provide a unique name to identify your trained model
            name="local_test",
            # Add a brief description to help users understand the purpose or functionality of this trained model
            description="a_description",
        ),
        # Specify the input and output columns for your dataset
        data_processing_parameters=TrainDataProcessingParametersProto(
            input_columns=[0, 1, 2, 3], output_columns=[4], timestamp_columns=[]
        ),
    ),
    inputs=MLTrainInputsProto(
        # Provide the specific dataset storage ID of the data you uploaded to TQ42
        data=DatasetStorageInfoProto(storage_id="47c6aba9-881b-4664-b714-dd9bb2f6bd5e")
    ),
)

with TQ42Client() as client:
    client.login()
    run = ExperimentRun.create(
        client=client,
        algorithm=AlgorithmProto.GENERIC_ML_TRAIN,
        # Provide the specific experiement ID your are running on TQ42
        experiment_id="b2004690-2a4b-479e-bafe-13a475bb0d69",
        compute=HardwareProto.SMALL,
        parameters=MessageToDict(env_msg, preserving_proto_field_name=True)
    )

    print(run.data)

    # Poll for the results
    result = run.poll()

    print(result.data)